phase 2

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install earthengine-api geemap

import ee
import geemap

ee.Authenticate()
ee.Initialize(project='uhi-prediction-487709')

india = ee.FeatureCollection("FAO/GAUL/2015/level0") \
            .filter(ee.Filter.eq('ADM0_NAME', 'India'))

In [ ]:
# laoding the saved model

import joblib

model = joblib.load("/content/drive/MyDrive/UHI Model Phases/uhi_model_final.pkl")

In [ ]:
import datetime

today = datetime.date.today()

# 16-day safety lag (for NDVI composite stability)
end_date = today - datetime.timedelta(days=16)

# 30-day training window before lag
start_date = end_date - datetime.timedelta(days=30)

start = start_date.strftime('%Y-%m-%d')
end = end_date.strftime('%Y-%m-%d')

print("Training window with 16-day lag:")
print("Start:", start)
print("End  :", end)

Loading datasets

In [ ]:
# MODIS LST

modis = ee.ImageCollection('MODIS/061/MOD11A2') \
          .filterDate(start, end) \
          .filterBounds(india) \
          .select('LST_Day_1km')

lst = modis.mean() \
           .multiply(0.02) \
           .subtract(273.15) \
           .rename('LST_Day_1km')

# ERA5 for air temp, wind and precipitation

era5 = ee.ImageCollection("ECMWF/ERA5_LAND/DAILY_AGGR") \
    .filterDate(start, end) \
    .filterBounds(india)

era_mean = era5.mean()

air_temp = era_mean.select('temperature_2m') \
    .subtract(273.15) \
    .rename('mean_2m_air_temp')

wind_speed = era_mean.expression(
    'sqrt(u*u + v*v)', {
        'u': era_mean.select('u_component_of_wind_10m'),
        'v': era_mean.select('v_component_of_wind_10m')
    }).rename('wind_speed')

precip = era_mean.select('total_precipitation_sum') \
    .rename('precipitation')

# MODIS NDVI

ndvi = ee.ImageCollection("MODIS/061/MOD13Q1") \
    .filterDate(start, end) \
    .filterBounds(india) \
    .select('NDVI') \
    .mean() \
    .multiply(0.0001) \
    .rename('NDVI')

# Urban Mask from MODIS -- static data

landcover = ee.ImageCollection("MODIS/061/MCD12Q1") \
    .first() \
    .select('LC_Type1')

urban_mask = landcover.eq(13).rename('Urban_Class')

# SRTM ELEVATION

elevation = ee.Image("USGS/SRTMGL1_003") \
              .select('elevation') \
              .clip(india)

# night lights from VIIRS

viirs_collection = ee.ImageCollection("NOAA/VIIRS/DNB/MONTHLY_V1/VCMCFG") \
    .filterBounds(india) \
    .sort('system:time_start', False)

viirs = viirs_collection.first().select('avg_rad').rename('night_lights')

# final image of all bands
feature_image = air_temp.addBands([wind_speed, precip, ndvi, urban_mask, elevation, viirs])
print("Feature stack bands:", feature_image.bandNames().getInfo())

In [ ]:
# Sample features
samples = feature_image.sample(
    region=india.geometry(),
    scale=5000,
    numPixels=3000,
    seed=42,
    geometries=False
)

# Export features
task = ee.batch.Export.table.toDrive(
    collection=samples,
    description='UHI_features_RT',
    folder='GEE_Exports',
    fileNamePrefix='UHI_features_RT',
    fileFormat='CSV'
)
task.start()
print("Features exporting")

In [ ]:
import pandas as pd

# Load exported features
df = pd.read_csv("/content/drive/MyDrive/GEE_Exports/UHI_features_RT.csv")
df = df.drop(columns=['system:index', '.geo'], errors='ignore')


base_features = ['mean_2m_air_temp', 'wind_speed', 'precipitation', 'NDVI',
                'Urban_Class', 'elevation', 'night_lights']
df_clean = df[base_features].dropna()

df_clean['Urban_Class'] = df_clean['Urban_Class'].astype(float)

# Create interactions
df_clean['temp_x_ndvi'] = df_clean['mean_2m_air_temp'] * df_clean['NDVI']
df_clean['urban_x_temp'] = df_clean['Urban_Class'] * df_clean['mean_2m_air_temp']
df_clean['ndvi_x_night'] = df_clean['NDVI'] * df_clean['night_lights']
df_clean['elev_x_temp'] = df_clean['elevation'] * df_clean['mean_2m_air_temp']
df_clean['wind_x_temp'] = df_clean['wind_speed'] * df_clean['mean_2m_air_temp']

# Urban_Class_1 creation (handles all-zero case)
df_clean['Urban_Class_1'] = (df_clean['Urban_Class'] == 1).astype(int)

# features must be same as in phase 1
model_features = [
    'mean_2m_air_temp', 'wind_speed', 'precipitation', 'NDVI',
    'elevation', 'night_lights', 'temp_x_ndvi', 'urban_x_temp',
    'ndvi_x_night', 'elev_x_temp', 'wind_x_temp', 'Urban_Class_1'
]

print("All 12 features created!")

# print("Urban pixels found:", df_clean['Urban_Class'].sum())

print(f"Final data shape: {df_clean[model_features].dropna().shape}")

# PREDICT UHI
df_final = df_clean[model_features].dropna()
X = df_final[model_features].values
uhi_pred = model.predict(X)
df_final['UHI_pred'] = uhi_pred

# RESULTS
print("\nNEAR REAL-TIME UHI RESULTS")
print(df_final['UHI_pred'].describe())

print(f"Hotspots (>3°C): {(df_final['UHI_pred'] > 3).sum():,} pixels")
print(f"Moderate (1-3°C): {(df_final['UHI_pred'].between(1,3)).sum():,} pixels")
print(f"UHI Mean: {df_final['UHI_pred'].mean():.2f}°C")

# Feature importance
importance_df = pd.DataFrame({
    'Feature': model_features,
    'Importance': model.feature_importances_
}).sort_values('Importance', ascending=False)

print("\nTop 5 Drivers:")
print(importance_df.head())

# Save complete results
output_path = "/content/drive/MyDrive/UHI reports/UHI_predictions_RT.csv"
df_final.to_csv(output_path, index=False)
print("\nSUCCESS! Saved")


phase 3

In [ ]:
!pip install schedule -q

In [ ]:
import geemap

# Creating interactive India UHI map
Map = geemap.Map(center=[20,78], zoom=5)

# Visualization parameters
uhi_vis = {'min':15, 'max':35, 'palette':['blue', 'green', 'yellow', 'orange', 'red']}
airtemp_vis = {'min':10, 'max':30, 'palette':['blue', 'cyan', 'yellow', 'red']}

# Add layers
Map.addLayer(lst, uhi_vis, '🗺️ Observed LST (°C)')
Map.addLayer(feature_image.select('mean_2m_air_temp'), airtemp_vis, '🌡️ Air Temp (°C)')
Map.addLayer(urban_mask.updateMask(urban_mask.eq(1)), {'palette':'purple'}, '🏙️ Urban Areas')

# legend
Map.add_colorbar(vis_params=uhi_vis, label='LST (°C)', layer_name='🗺️ Observed LST (°C)')
Map.add_colorbar(vis_params=airtemp_vis, label='Air Temp (°C)', layer_name='🌡️ Air Temp (°C)')

print("Zoom to urban areas to see predictions.")
Map

In [ ]:
# UHI action dashboard

alerts = df_final[df_final['UHI_pred'] > 3][['mean_2m_air_temp', 'NDVI', 'elevation', 'night_lights', 'UHI_pred']]
alerts['Priority'] = pd.cut(alerts['UHI_pred'], bins=[3,5,8,12], labels=['High','Very High','Extreme'])

print("UHI WEEKLY ACTION DASHBOARD\n")

print(f"Analysis Period: {start} to {end}")
print(f"National UHI Mean: {df_final['UHI_pred'].mean():.2f}°C")
print(f"CRITICAL HOTSPOTS: {len(alerts):,} pixels")
print(f"Urban Coverage: {df_final['Urban_Class_1'].sum():,} pixels\n")

print("TOP 5 HOTSPOTS:")
print(alerts.nlargest(5, 'UHI_pred')[['UHI_pred', 'Priority', 'NDVI', 'mean_2m_air_temp']].round(2))

# Export alerts
alerts.round(2).to_csv(f"/content/drive/MyDrive/UHI reports/UHI_ALERTS_{end.replace('-','')}.csv")
print(f"\nAlerts exported: UHI_ALERTS_{end.replace('-','')}.csv")

IMD validation

In [ ]:
# Major Indian cities as IMD station proxies
cities = ee.FeatureCollection([
    ee.Feature(ee.Geometry.Point([77.1025, 28.7041]), {'city': 'Delhi'}),
    ee.Feature(ee.Geometry.Point([72.8777, 19.0760]), {'city': 'Mumbai'}),
    ee.Feature(ee.Geometry.Point([77.5946, 12.9716]), {'city': 'Bangalore'}),
    ee.Feature(ee.Geometry.Point([80.2707, 13.0827]), {'city': 'Chennai'}),
    ee.Feature(ee.Geometry.Point([88.3639, 22.5726]), {'city': 'Kolkata'}),
    ee.Feature(ee.Geometry.Point([78.4867, 17.3850]), {'city': 'Hyderabad'}),
    ee.Feature(ee.Geometry.Point([75.7139, 15.3173]), {'city': 'Pune'}),
    ee.Feature(ee.Geometry.Point([73.8567, 18.5204]), {'city': 'Ahmedabad'}),
    ee.Feature(ee.Geometry.Point([72.8369, 19.0134]), {'city': 'Surat'}),
    ee.Feature(ee.Geometry.Point([75.8577, 26.9124]), {'city': 'Lucknow'}),
])

# Sample at city centers (500m accuracy)
city_features = feature_image.sampleRegions(
    collection=cities,
    properties=['city'],
    scale=500,
    geometries=True
)

task = ee.batch.Export.table.toDrive(
    collection=city_features,
    description='IMD_Cities_UHI',
    folder='GEE_Exports',
    fileNamePrefix='IMD_Cities_UHI',
    fileFormat='CSV'
)
task.start()

In [ ]:
# Load city data
city_df = pd.read_csv("/content/drive/MyDrive/GEE_Exports/IMD_Cities_UHI.csv")

# Clean & predict
city_clean = city_df[['city', 'mean_2m_air_temp', 'wind_speed', 'precipitation',
                     'NDVI', 'Urban_Class', 'elevation', 'night_lights']].dropna()
city_clean['Urban_Class'] = city_clean['Urban_Class'].astype(float)

# Recreate interactions
city_clean['temp_x_ndvi'] = city_clean['mean_2m_air_temp'] * city_clean['NDVI']
city_clean['urban_x_temp'] = city_clean['Urban_Class'] * city_clean['mean_2m_air_temp']
city_clean['ndvi_x_night'] = city_clean['NDVI'] * city_clean['night_lights']
city_clean['elev_x_temp'] = city_clean['elevation'] * city_clean['mean_2m_air_temp']
city_clean['wind_x_temp'] = city_clean['wind_speed'] * city_clean['mean_2m_air_temp']
city_clean['Urban_Class_1'] = (city_clean['Urban_Class'] == 1).astype(int)

# Predict UHI
model_features = ['mean_2m_air_temp', 'wind_speed', 'precipitation', 'NDVI',
                 'elevation', 'night_lights', 'temp_x_ndvi', 'urban_x_temp',
                 'ndvi_x_night', 'elev_x_temp', 'wind_x_temp', 'Urban_Class_1']

X_cities = city_clean[model_features]
city_uhi = model.predict(X_cities)

# Results table
results = pd.DataFrame({
    'City': city_clean['city'],
    'UHI_Predicted': city_uhi.round(2),
    'Air_Temp': city_clean['mean_2m_air_temp'].round(1),
    'NDVI': city_clean['NDVI'].round(2),
    'Urban': city_clean['Urban_Class_1'],
    'Elevation': city_clean['elevation'].round(0)
})

print("\nMAJOR CITIES UHI VALIDATION\n")
print(results.sort_values('UHI_Predicted', ascending=False).to_string(index=False))

print(f"\nVALIDATION METRICS")
print(f"National UHI Mean: {df_final['UHI_pred'].mean():.2f}°C")
print(f"City UHI Mean:    {city_uhi.mean():.2f}°C")
print(f"Difference:       ±{abs(df_final['UHI_pred'].mean() - city_uhi.mean()):.2f}°C")
print(f"Hot cities (>3°C): {(city_uhi > 3).sum()} / {len(city_uhi)}")

# Export publication table
results.to_csv("/content/drive/MyDrive/UHI reports/City_UHI_Validation.csv", index=False)
print("\nPUBLICATION TABLE SAVED!")